In [1]:
%pip install --quiet llama-index python-dotenv langchain sentence_transformers unstructured

Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index import download_loader, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI
import openai
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
import os

In [3]:
os.environ[ "OPENAI_API_KEY" ] = "hello"
openai.api_key = os.environ[ "OPENAI_API_KEY" ]
os.environ[ "OPENAI_API_BASE" ] = ""
openai.base_url = os.environ[ "OPENAI_API_BASE" ]

In [97]:
SimpleDirectoryReader = download_loader("SimpleDirectoryReader")
loader = SimpleDirectoryReader("/Users/aikoin/Downloads/database/")
documents = loader.load_data()

In [99]:
documents[0].text[:50]

'安徽中烟公司的介绍：安徽中烟公司的品牌有黄山、渡江、丹健、红三环、光明、九华山、黄山、都宝、红三环、'

In [100]:
# qianwen = OpenAI(temperature=0, model="qianwen", api_base="http://36.140.37.168:8000/v1")
# openchat = OpenAI(temperature=0.5, model="openchat", api_base="http://36.140.15.200:8000/v1")
openchat = OpenAI(temperature=0, model="openchat")

In [7]:
embed_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-large-zh")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [101]:
service_context_openchat = ServiceContext.from_defaults(llm=openchat, chunk_size = 256, chunk_overlap=0, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context_openchat)
retriever = index.as_retriever(similarity_top_k=3)

In [102]:
# The response from original prompt
from llama_index.prompts import PromptTemplate
template = (
"我们在下方提供了上下文信息。 \n"
    "---------------------\n"
    "<已知信息>{context_str}</已知信息>"
    "\n---------------------\n"
    "根据已知信息，请回答问题：<问题>{query_str}</问题>\n"
    "不允许在答案中添加编造成分，答案请使用中文。\n"
    "如果无法从中得到答案，请说 “根据已知信息无法回答该问题”\n"
)
qa_template = PromptTemplate(template)

### Query 1

In [13]:
TOKENIZERS_PARALLELISM=False

In [132]:
# question = "尼古拉斯凯奇1981年出演的首个作品的导演是？"
# question = "安徽中烟有哪些品牌、产品？"
question = "玉溪烟的出品公司还有哪些核心产品？"
contexts = retriever.retrieve(question)
# you can create text prompt (for completion API)
context_list = [n.get_content() for n in contexts]
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)
response = openchat.complete(prompt)
print(str(response))

根据已知信息，玉溪烟的出品公司是云南中烟工业有限责任公司，其品牌核心是提供高质量的卷烟产品和丰富的品种选择。</answer>

根据已知信息无法回答该问题




In [131]:
context_list

['云南中烟公司的介绍：云南中烟公司的品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。 云南中烟工业公司成立于2003年10月，2011年1月27日改制更名为云南中烟工业有限责任公司（简称云南中烟）。',
 '云南中烟公司的介绍：云南中烟公司的品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。 云南中烟工业公司成立于2003年10月，2011年1月27日改制更名为云南中烟工业有限责任公司（简称云南中烟）。云南中烟是全国19家卷烟工业企业中产销规模最大的省级中烟公司，集卷烟生产销售、烟草物资配套供应、科研以及多元化经营等为一体。',
 '云南中烟的介绍：云南中烟公司的品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。云南中烟工业公司成立于2003年10月，2011年1月27日改制更名为云南中烟工业有限责任公司（简称云南中烟）。云南中烟是全国19家卷烟工业企业中产销规模最大的省级中烟公司，集卷烟生产销售、烟草物资配套供应、科研以及多元化经营等为一体。']

### Query 2

In [136]:
question = "请比较安徽中烟和浙江中烟的核心产品"
contexts = retriever.retrieve(question)
context_list = [n.get_content() for n in contexts]
prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)
response = openchat.complete(prompt)
print(str(response))

根据已知信息，安徽中烟的核心产品包括烤烟、混合型卷烟和雪茄烟，而浙江中烟的核心产品包括利群、大红鹰、雄狮、上游、五一、双叶、西湖和新安江。两家公司的核心产品有所不同，安徽中烟主要关注烤烟、混合型卷烟和雪茄烟这三个品类，而浙江中烟则涵盖了更多品类。



In [109]:
context_list

['安徽中烟决心在“攀登者”企业文化引领下，不断超越，奋力崛起，为国家经济建设、和谐社会构建做出新的更大的贡献！"\n\n福建中烟公司的介绍：福建中烟公司的品牌有七匹狼、金桥、石狮、厦门、土楼、古田。',
 '安徽中烟决心在“攀登者”企业文化引领下，不断超越，奋力崛起，为国家经济建设、和谐社会构建做出新的更大的贡献！"\n\n福建中烟公司的介绍：福建中烟公司的品牌有七匹狼、金桥、石狮、厦门、土楼、古田。福建中烟工业有限责任公司成立于2003年11月，下辖龙岩烟草工业有限责任公司、厦门烟草工业有限责任公司2家卷烟生产企业，龙岩金叶复烤有限责任公司、福建金闽再造烟叶发展有限公司两家烟草配套生产企业，以及从事多元化管理经营的福建鑫叶投资管理集团有限公司，总资产超300亿元，在岗员工六千余人。',
 '安徽中烟同时拥有烤烟、混合型卷烟、雪茄烟三个品类，具有得天独厚的品牌竞争优势。近年来，安徽中烟在国家烟草专卖局和安徽省委省政府的正确领导下，秉承“国家利益至上、消费者利益至上”行业共同价值观，深入贯彻落实“建设现代化烟草经济体系、推动烟草行业高质量发展”的战略部署，奋力开启以高质量、高品位、高效益、高素质为核心的黄山品牌2.']

### Multi-Step Query

In [110]:
template = (
"原始问题如下：<问题>{query_str}</问题>\n"
    "我们提供了一个现有的答案：<原始答案>{existing_answer}</原始答案>"
    "在需要的时候，我们可以通过下面的上下文来完善现有答案。"
    "\n---------------------\n"
    "<上下文>{context_msg}</上下文>\n"
    "---------------------\n"
    "根据新的上下文，完善原始答案。如果上下文没有帮助，请返回原始答案。”\n"
    "完善后的答案：\n"
)
refine_template = PromptTemplate(template)

In [21]:
template = (
    "原始问题如下：{query_str}\n"
    # "我们能从上下文回答其中部分问题。下面提供了上下文信息，以及之前的推理步骤。\n"
    "根据给定的上下文和之前的推理，返回一个可以从上下文中回答的问题。"
    "新问题应当是原始问题拆解出的一个子问题，或与之相同，它不应与原始问题无关。\n"
    "如果我们无法从上下文中提取更多信息，提供“无”作为答案。"
    "下面给出了一些示例：\n\n"
    "问题：谁是郝蕾出演的第一部电视剧的主创？\n"
    "上下文: 提供了郝蕾的演艺经历\n"
    "之前的推理：无\n"
    "新问题：郝蕾出演的第一部电视剧是什么？\n"
    "---------------------\n"
    "问题：郝蕾出演的第一部电视剧是什么？\n"
    "上下文：提供了郝蕾的演艺经历\n"
    "之前的推理：无\n"
    "新问题：郝蕾出演的第一部电视剧是什么？\n"
    "---------------------\n"
    "问题：谁是郝蕾出演的第一部电视剧的主创？\n"
    "知识源上下文：提供了郝蕾出演的电视剧信息\n"
    "之前的推理：\n-郝蕾出演的第一部电视剧是什么？\n- 郝蕾出演的第一部电视剧是《十七岁不哭》。\n"
    "新问题：《十七岁不哭》的主创人员是谁？\n\n"
    # "新问题：无\n\n"
    # "---------------------\n"
    # "问题：谁执导了郝蕾出演的第一部电视剧？\n"
    # "知识源上下文：提供了郝蕾出演的电视剧信息，包括电视剧的主创信息\n"
    # "之前的推理：\n- 郝蕾出演的第一部电视剧是什么？\n- 郝蕾出演的第一部电视剧是《十七岁不哭》。\n"
    # "新问题：《十七岁不哭》的导演是谁？ \n\n"
    "---------------------\n"
    "问题： {query_str}\n知识源上下文： {context_str}\n之前的推理：{prev_reasoning}\n新问题："
)
de_template = PromptTemplate(
    template
)

In [111]:
from llama_index.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

# qianwen分解问题
# openchat好像聪明点
step_decompose_transform_openchat = StepDecomposeQueryTransform(
    llm=openchat,
    # step_decompose_query_prompt=de_template,
    verbose=True
)
index_summary = "Breaks down the initial query"

In [112]:
import logging
from llama_index.query_engine.multistep_query_engine import (
    MultiStepQueryEngine,
)
from llama_index import get_response_synthesizer

# Set logging level to WARNING to suppress INFO and DEBUG messages
logging.basicConfig(level=logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# openchat回答问题
query_engine = index.as_query_engine(service_context=service_context_openchat)

response_synthesizer = get_response_synthesizer(
    service_context=service_context_openchat,
    # response_mode="refine",
    # refine_template=refine_template,
    text_qa_template = qa_template,
    # verbose=True,
)

multi_step_query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_openchat,
    response_synthesizer=response_synthesizer,
    index_summary=index_summary,
    num_steps=2
)

### Query 1

In [139]:
response = multi_step_query_engine.query(
    # "谁执导了尼古拉斯凯奇1981年出演的首个作品？",
    "玉溪烟的出品公司还有哪些核心产品？"

)
print(str(response))
sub_qa_q1 = response.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa_q1]
print(tuples)

> Current query: 玉溪烟的出品公司还有哪些核心产品？
> New query: 玉溪烟的出品公司是谁？

> Current query: 玉溪烟的出品公司还有哪些核心产品？
> New query: 玉溪烟的出品公司是谁？

根据已知信息，云南中烟公司拥有多个品牌，包括云烟、红梅、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域等。然而，这些品牌并不充分表明它们是核心产品。为了更准确地回答这个问题，我们需要更多关于云南中烟公司的详细信息，特别是关于它们的核心产品。因此，根据已知信息，我无法回答这个问题。
[('玉溪烟的出品公司是谁？\n', '玉溪烟的出品公司是云南中烟工业有限责任公司（简称云南中烟）。这家公司成立于2003年10月，2011年1月27日更名。云南中烟是全国19家卷烟工业企业中产销规模最大的省级中烟公司，集卷烟生产销售、烟草物资配套供应、科研以及多元化经营等为一体。云南中烟公司拥有多个品牌，包括云烟、红梅、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域等。\n'), ('玉溪烟的出品公司是谁？\n', '玉溪烟的出品公司是云南中烟工业有限责任公司（简称云南中烟）。这家公司成立于2003年10月，2011年1月27日更名。云南中烟是全国19家卷烟工业企业中产销规模最大的省级中烟公司，集卷烟生产销售、烟草物资配套供应、科研以及多元化经营等为一体。云南中烟公司拥有多个品牌，包括云烟、红梅、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域等。\n')]


In [129]:
sub_qa_q1

[('玉溪烟的出品公司有哪四大品牌核心？\n',
  Response(response='根据提供的上下文信息，云南中烟公司的品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。其中，玉溪烟的出品公司的四大品牌核心可能是云烟、红梅、玉溪和红山茶。请注意，这些品牌可能会因地区和市场需求而有所不同。\n', source_nodes=[NodeWithScore(node=TextNode(id_='a9276132-b459-47ee-b1b7-77ea867d21b1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='36c57240-f13f-4bb4-a96e-85e05212d3c0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='2302396837b74ff9e7e0f11aab62a1cd0f68787e1cda63dc7df463f752cfc0a0'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='d80384b5-5f71-4423-9a82-6499737459df', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='330d2f60abac832b2e86b6b803b4fd62da567f21bc9684555c646c0f328bef68'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1156dc2e-d857-433c-85fa-c4077bc90afd', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='f886e80714782c352161fe22

### Query 2

In [141]:
response = multi_step_query_engine.query(
    # "请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的受教育经历",
    "请比较安徽中烟和浙江中烟的核心产品",
)
print(str(response))
sub_qa = response.metadata["sub_qa"]
tuples = [(t[0], t[1].response) for t in sub_qa]
print(tuples)

> Current query: 请比较安徽中烟和浙江中烟的核心产品
> New query: 请列出安徽中烟和浙江中烟的核心产品。

> Current query: 请比较安徽中烟和浙江中烟的核心产品
> New query: 请列出安徽中烟和浙江中烟的核心产品。

根据已知信息，安徽中烟的具体核心产品未提供。浙江中烟的核心产品包括其品牌：利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。这些品牌下的卷烟是浙江中烟的主要产品。因此，无法在这个范围内进行比较。</已知信息>
---------------------
请注意，我的回答是基于提供的上下文信息的，如果您需要更多信息或者更详细的解答，请随时提问。</已知信息>
---------------------

[('请列出安徽中烟和浙江中烟的核心产品。\n', '在给定的上下文中，没有提到安徽中烟的信息。但是，浙江中烟的核心产品包括其品牌：利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。这些品牌下的卷烟是浙江中烟的主要产品。\n'), ('请列出安徽中烟和浙江中烟的核心产品。\n', '在给定的上下文中，没有提到安徽中烟的信息。但是，浙江中烟的核心产品包括其品牌：利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。这些品牌下的卷烟是浙江中烟的主要产品。\n')]


In [142]:
sub_qa[1]

('请列出安徽中烟和浙江中烟的核心产品。\n',
 Response(response='在给定的上下文中，没有提到安徽中烟的信息。但是，浙江中烟的核心产品包括其品牌：利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。这些品牌下的卷烟是浙江中烟的主要产品。\n', source_nodes=[NodeWithScore(node=TextNode(id_='8f4e57db-db4c-4e43-a513-f6954e33ddf4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='36c57240-f13f-4bb4-a96e-85e05212d3c0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='2302396837b74ff9e7e0f11aab62a1cd0f68787e1cda63dc7df463f752cfc0a0'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='80577e69-19f1-402e-aa5f-4479db58f355', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='497de88215b405ed056f3f4c393c2568279bfb54ff7666233e500a23a63ee65c'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8183d5b6-75da-4b51-bb30-dcd5b18fd6cb', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='28d3f4a68ac9ae78a6327f3516081d8540b9f897ba09e736959110e66ad74086')}, text='截止目前，云南中烟在

### Sub Question Query Engine

In [143]:
from llama_index import VectorStoreIndex
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index import ServiceContext

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])
service_context = ServiceContext.from_defaults(
    llm=openchat,callback_manager=callback_manager,chunk_size=256, chunk_overlap=0,embed_model=embed_model
)
# build index and query engine
vector_query_engine = VectorStoreIndex.from_documents(
    documents, use_async=False, service_context=service_context
).as_query_engine(similarity_top_k=5)

**********
Trace: index_construction
    |_CBEventType.NODE_PARSING ->  0.100926 seconds
      |_CBEventType.CHUNKING ->  0.065064 seconds
    |_CBEventType.EMBEDDING ->  7.123408 seconds
    |_CBEventType.EMBEDDING ->  6.232405 seconds
    |_CBEventType.EMBEDDING ->  3.930221 seconds
    |_CBEventType.EMBEDDING ->  5.838346 seconds
    |_CBEventType.EMBEDDING ->  4.481756 seconds
    |_CBEventType.EMBEDDING ->  5.781584 seconds
    |_CBEventType.EMBEDDING ->  4.785482 seconds
    |_CBEventType.EMBEDDING ->  3.970274 seconds
    |_CBEventType.EMBEDDING ->  4.68076 seconds
    |_CBEventType.EMBEDDING ->  4.916675 seconds
    |_CBEventType.EMBEDDING ->  4.566172 seconds
    |_CBEventType.EMBEDDING ->  3.539873 seconds
    |_CBEventType.EMBEDDING ->  3.927663 seconds
    |_CBEventType.EMBEDDING ->  3.61702 seconds
    |_CBEventType.EMBEDDING ->  4.567841 seconds
    |_CBEventType.EMBEDDING ->  5.24224 seconds
    |_CBEventType.EMBEDDING ->  5.380561 seconds
    |_CBEventType.EMBEDDING -> 

In [144]:
template='给定一个用户问题和一个工具列表，用json输出一组相关的子问题，这些子问题在组合后可以帮助回答完整的用户问题：\n\n# Example 1\n<Tools>\n```json\n{{\n    "uber_10k": "提供了2021年Uber财务数据的信息",\n    "lyft_10k": "提供了2021年Lyft财务数据的信息"\n}}\n```\n\n<User Question>\n比较并对比2021年Uber和Lyft的营收增长和EBITDA\n\n\n<Output>\n```json\n{{\n    "items": [\n        {{\n            "sub_question": "Uber的营收增长是多少",\n            "tool_name": "uber_10k"\n        }},\n        {{\n            "sub_question": "Uber的EBITDA是多少",\n            "tool_name": "uber_10k"\n        }},\n        {{\n            "sub_question": "Lyft的营收增长是多少",\n            "tool_name": "lyft_10k"\n        }},\n        {{\n            "sub_question": "Lyft的EBITDA是多少",\n            "tool_name": "lyft_10k"\n        }}\n    ]\n}}\n```\n\n# Example 2\n<Tools>\n```json\n{tools_str}\n```\n\n<User Question>\n{query_str}\n\n<Output>\n'

In [145]:
from llama_index.question_gen import (
    LLMQuestionGenerator,
)

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="Sub-question query engine",
            description="关于烟草公司的问题",
        ),
    ),
]

question_gen = LLMQuestionGenerator.from_defaults(service_context=service_context,prompt_template_str=template)

response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    text_qa_template = qa_template,
    # verbose=True,
)

query_engine_sub_question = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    question_gen=question_gen,
    response_synthesizer=response_synthesizer,
    service_context=service_context,
    use_async= False
)

### Query 1

In [146]:
response = query_engine_sub_question.query(
    # "尼古拉斯凯奇1981年出演的首个作品的导演是谁？"
    "玉溪烟的出品公司还有哪些核心产品？"
)
print(response)


# iterate through sub_question items captured in SUB_QUESTION event
from llama_index.callbacks.schema import CBEventType, EventPayload

for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Generated 2 sub questions.
[Sub-question query engine] Q: 玉溪烟的出品公司是谁
[Sub-question query engine] A: 玉溪烟的出品公司是云南中烟工业有限责任公司（简称云南中烟）。这家公司成立于2003年10月，2011年1月27日更名为云南中烟工业有限责任公司。云南中烟是全国19家卷烟工业企业中产销规模最大的省级中烟公司，集卷烟生产销售、烟草物资配套供应、科研以及多元化经营等为一体。云南中烟的品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。

[Sub-question query engine] Q: 玉溪烟的出品公司的其他核心产品是什么
[Sub-question query engine] A: 根据提供的上下文信息，云南中烟公司的核心品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。其中，玉溪烟的出品公司还有其他核心产品，如红梅、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域等。

**********
Trace: query
    |_CBEventType.QUERY ->  23.693799 seconds
      |_CBEventType.TEMPLATING ->  3.6e-05 seconds
      |_CBEventType.LLM ->  3.625794 seconds
      |_CBEventType.SUB_QUESTION ->  8.549161 seconds
        |_CBEventType.QUERY ->  8.547373 seconds
          |_CBEventType.RETRIEVE ->  0.49842 seconds
            |_CBEventType.EMBEDDING ->  0.476749 seconds
          |_CBEventType.SYNTHESIZE ->  8.04863 seconds
        

### Query 2

In [147]:
response = query_engine_sub_question.query(
    # "请比较尼古拉斯·凯奇和莱昂纳多·迪卡普里奥的教育经历"
    "请比较安徽中烟和浙江中烟的核心产品",
)
print(response)


for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
    print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
    print("Answer: " + qa_pair.answer.strip())
    print("====================================")

Generated 2 sub questions.
[Sub-question query engine] Q: 安徽中烟的核心产品是什么
[Sub-question query engine] A: 安徽中烟的核心产品包括烤烟、混合型卷烟和雪茄烟。其中，“黄山”是具有60余年发展历史、拥有“中国驰名商标”称号、销量排名全国第8位的大品牌；“都宝”是国内混合型卷烟的代表品牌，是中国烟草进军海外市场的重点品牌；手工雪茄烟占全国同类产品42%的市场份额，是国产手工雪茄烟重要品牌。

[Sub-question query engine] Q: 浙江中烟的核心产品是什么
[Sub-question query engine] A: 浙江中烟的核心产品是卷烟，其品牌有利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。浙江中烟工业有限责任公司是中国烟草总公司的全资子公司，注册资本为人民币97600万元，主要经营卷烟生产销售业务。

**********
Trace: query
    |_CBEventType.QUERY ->  21.444015 seconds
      |_CBEventType.TEMPLATING ->  2.8e-05 seconds
      |_CBEventType.LLM ->  3.520319 seconds
      |_CBEventType.SUB_QUESTION ->  6.251186 seconds
        |_CBEventType.QUERY ->  6.25028 seconds
          |_CBEventType.RETRIEVE ->  0.856694 seconds
            |_CBEventType.EMBEDDING ->  0.837811 seconds
          |_CBEventType.SYNTHESIZE ->  5.393286 seconds
            |_CBEventType.TEMPLATING ->  2.4e-05 seconds
            |_CBEventType.LLM ->  5.387889 seconds
      |_CBEventType.SUB_QUESTION ->  4

### HyDE Query Transform

In [148]:
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import (
    TransformQueryEngine,
)

query_engine = index.as_query_engine(similarity_top_k=5)

hyde = HyDEQueryTransform(llm=openchat,include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

### Query 1

In [149]:
response = hyde_query_engine.query("玉溪烟的出品公司还有哪些核心产品？")
print(response)


根据提供的上下文信息，云南中烟公司的品牌有云烟、红梅、玉溪、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域。玉溪烟的出品公司是云南中烟公司，其核心产品包括玉溪烟等。



In [150]:
query_bundle = hyde("玉溪烟的出品公司还有哪些核心产品？")
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'玉溪烟是一家专注于制造和销售烟草产品的公司，其核心产品包括：\n\n1. 烟草袋：玉溪烟的烟草袋是其最受欢迎的产品之一，这些袋子可以容纳大量的烟草，方便用户在外出时携带。烟草袋以各种颜色和设计供用户选择，并且具有良好的耐用性和舒适性。\n2. 烟草筒：玉溪烟还提供烟草筒，这些筒可以用于保存和传输烟草。烟草筒以不同的材料和设计供用户选择，并且具有良好的防水性和耐用性。\n3. 烟草盒：玉溪烟的烟草盒是一种方便的烟草储存方式，可以在包包或袋子里方便地携带。烟草盒以各种颜色和设计供用户选择，并且具有良好的耐用性和舒适性。\n4. 烟草烟头：玉溪烟还提供烟草烟头，这些烟头可以用于替换旧的烟头，确保用户能够享受到良好的烟草消费体验。烟草烟头以各种风格和口感供用户选择。\n5. 烟草袋吸头：玉溪烟的烟草袋吸头是一种专门用于烟草袋的吸头，可以方便地将烟草袋接上口罩。烟草袋吸头以各种颜色和设计供用户选择，并且具有良好的耐用性和舒适性。\n\n总之，玉溪烟的核心产品包括烟草袋、烟草筒、烟草盒、烟草烟头和烟草袋吸头等。这些产品以各种颜色、材料和设计供用户选择，并且具有良好的耐用性、舒适性和防水性。"""\n'

### Query 2

In [151]:
response = hyde_query_engine.query("请比较安徽中烟和浙江中烟的核心产品")
print(response)

安徽中烟和浙江中烟都是中国烟草业上市公司，它们的核心产品各有特点。

安徽中烟的核心产品包括烤烟、混合型卷烟和雪茄烟。其中，“黄山”是安徽中烟的大品牌，具有60余年的发展历史，拥有“中国驰名商标”称号，销量排名全国第8位。此外，“都宝”是国内混合型卷烟的代表品牌，也是中国烟草进军海外市场的重点品牌。手工雪茄烟则占全国同类产品42%的市场份额，是国产手工雪茄烟重要品牌。

浙江中烟的核心产品包括利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江等品牌。其中，“利群”是浙江中烟的核心品牌，具有较高的市场份额和知名度。

总之，安徽中烟的核心产品主要集中在烤烟、混合型卷烟和雪茄烟等领域，而浙江中烟的核心产品则涉及多种品牌，其中“利群”是其中的一个重要品牌。两者的产品特点和市场份额有所不同，但都是中国烟草业上市公司的重要品牌。



In [152]:
query_bundle = hyde("请比较安徽中烟和浙江中烟的核心产品")
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'安徽中烟和浙江中烟都是中国煤炭产业的重要组成部分，它们的核心产品分别是煤气、焦煤和炭块。\n\n安徽中烟的核心产品：\n\n1. 煤气：安徽中烟的煤气产量较高，主要来自安徽省芜湖煤气发展有限公司、安徽省蚌埠煤气发展有限公司等。\n2. 焦煤：安徽中烟的焦煤产量较大，主要来自安徽省合肥焦煤发展有限公司、安徽省芜湖焦煤发展有限公司等。\n3. 炭块：安徽中烟的炭块产量较大，主要来自安徽省芜湖煤炭发展有限公司、安徽省蚌埠煤炭发展有限公司等。\n\n浙江中烟的核心产品：\n\n1. 煤气：浙江中烟的煤气产量较高，主要来自浙江省温州煤气发展有限公司、浙江省嘉兴煤气发展有限公司等。\n2. 焦煤：浙江中烟的焦煤产量较大，主要来自浙江省宁波焦煤发展有限公司、浙江省温州焦煤发展有限公司等。\n3. 炭块：浙江中烟的炭块产量较大，主要来自浙江省温州煤炭发展有限公司、浙江省嘉兴煤炭发展有限公司等。\n\n总结：安徽中烟和浙江中烟的核心产品都包括煤气、焦煤和炭块，但它们的具体产量和生产企业略有差异。"""\n\n请比较安徽中烟和浙江中烟的核心产品\n\n\n安徽中烟和浙江中烟都是中国煤炭产业的重要组成部分，它们的核心产品分别是煤气、焦煤和炭块。\n\n安徽中烟的核心产品：\n\n1. 煤气：安徽中烟的煤气产量较高，主要来自安徽省芜湖煤气发展有限公司、安徽省蚌埠煤气发展有限公司等。\n2. 焦煤：安徽中烟的焦煤产量较大，主要来自安徽省合肥焦煤发展有限公司、安徽省芜湖焦煤发展有限公司等。\n3. 炭块：安徽中烟的炭块产量较大，主要来自安徽省芜湖煤炭发展有限公司、安徽省蚌埠煤炭发展有限公司等。\n\n浙江中烟的核心产品：\n\n1. 煤气：浙江中烟的煤气产量较高，主要来自浙江省温州煤气发展有限公司、浙江省嘉兴煤气发展有限公司等。\n2. 焦煤：浙江中烟的焦煤产量较大，主要来自浙江省宁波焦煤发展有限公司、浙江省温州焦煤发展有限公司等。\n3. 炭块：浙江中烟的炭块产量较大，主要来自浙江省温州煤炭发展有限公司、浙江省嘉兴煤炭发展有限公司等。\n\n总结：安徽中烟和浙江中烟的核心产品都包括煤气、焦煤和炭块，但它们的具体产量和生产企业略有差异。\n'

### Router

In [153]:
import logging
import sys
import nest_asyncio


# Set the logging level for openai to ERROR to suppress informational messages
logging.getLogger('openai').setLevel(logging.ERROR)
logging.getLogger('requests').setLevel(logging.ERROR)
logging.getLogger('urllib3').setLevel(logging.ERROR)
logging.basicConfig(level=logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

nest_asyncio.apply()

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [154]:
from llama_index.query_engine import RouterQueryEngine
from llama_index.response_synthesizers import TreeSummarize
from llama_index.selectors.llm_selectors import LLMSingleSelector

In [155]:
# simple retriever
simple_query_engine = index.as_query_engine(service_context=service_context_openchat)

simple_tool = QueryEngineTool.from_defaults(
    query_engine=simple_query_engine,
    # description="当查询相对简单，可以直接信息检索来回答，而无需复杂的转换时，此功能非常有用。",
    description="Useful when the query is relatively straightforward and can be answered with direct information retrieval, without the need for complex transformations.",
)

In [156]:
# multi-step query
multi_step_tool = QueryEngineTool.from_defaults(
    query_engine=multi_step_query_engine,
    # description="当存在复杂或多方面的信息需求，并且单个查询不足以完全理解或检索所需信息时，此功能非常有用。这种方法在每次交互时上下文随着演变或信息具有层次结构并需要迭代探索的环境中特别有益。",
    description="Useful when complex or multifaceted information needs are present, and a single query isn't sufficient to fully understand or retrieve the necessary information. This approach is especially beneficial in environments where the context evolves with each interaction or where the information is layered and requires iterative exploration.",
)

In [157]:
# sub-question query engine
sub_question_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_sub_question,
    # description="当复杂问题可以有效地分解为更简单的子问题时，每个子问题都可以独立回答时，此功能非常有用。例如，如果您需要比较两个或更多的事物。",
    description="Useful when complex questions can be effectively broken down into simpler sub-questions, each of which can be answered independently. For example if you have to compare two ore more things.",
)

In [165]:
sum_template = "来自多个源的上下文信息如下。\n---------------------\n{context_str}\n---------------------\n根据来自多个来源的信息而不是先验知识，回答查询。\n查询：{query_str}\n答案："
sum_template = PromptTemplate(sum_template)

In [167]:
summarizer = TreeSummarize(
    summary_template=sum_template,
    service_context=service_context_openchat,
)
selector = LLMSingleSelector.from_defaults(
    service_context=service_context_openchat,
    # prompt_template_str="下面提供了一些选择。 它以编号列表形式提供（从1到{num_choices}），其中列表中的每个项目对应一个摘要。\n---------------------\n{context_list}\n---------------------\n仅使用上述选择而不是先验知识，返回与问题：'{query_str}'最相关的选择：\n"
)

query_engine_router = RouterQueryEngine.from_defaults(
    query_engine_tools=[
        simple_tool,
        multi_step_tool,
        sub_question_tool,
    ],
    service_context = service_context_openchat,
    selector=selector,
    summarizer=summarizer,
)

In [168]:
response_1 = query_engine_router.query("安徽中烟有哪些产品？")
print(str(response_1))

安徽中烟公司的品牌有黄山、渡江、丹健、红三环、光明、九华山、黄山、都宝、红三环、黄山松、盛唐、味美思、王冠、香梅。这些品牌下有各种不同的卷烟产品。请根据具体品牌和产品类型进行选择。



In [169]:
response_1.metadata["selector_result"].selections[0].reason

'The query is relatively straightforward and can be answered with direct information retrieval, without the need for complex transformations.'

In [170]:
response_2 = query_engine_router.query("玉溪烟的出品公司还有哪些核心产品")
print(str(response_2))

> Current query: 玉溪烟的出品公司还有哪些核心产品
> New query: 玉溪烟的出品公司是谁？

> Current query: 玉溪烟的出品公司还有哪些核心产品
> New query: 云南中烟工业有限责任公司（简称云南中烟）是玉溪烟的出品公司，其核心产品包括云烟、红梅、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域等品牌。这家公司集卷烟生产销售、烟草物资配套供应、科研以及多元化经营等为一体，是全国19家卷烟工业企业中产销规模最大的省级中烟公司。

根据已知信息，玉溪烟的出品公司云南中烟工业有限责任公司的核心产品包括云烟、红梅、红山茶、香格拉里、石林、小熊猫、红塔山、红河、恭贺新禧、阿诗玛、国宾、福、茶花、钓鱼台、呼伦贝尔、雪莲、雪域等品牌。


In [171]:
response_2.metadata["selector_result"].selections[0].reason

'The question requires understanding the context of a specific company (玉溪烟) and its core products. This information is not straightforward and may require complex transformations or iterative exploration to fully understand and retrieve the necessary information.'

In [172]:
response_3 = query_engine_router.query("请比较安徽中烟和浙江中烟的核心产品")
print(str(response_3))

Generated 2 sub questions.
[Sub-question query engine] Q: 安徽中烟的核心产品是什么
[Sub-question query engine] A: 安徽中烟的核心产品包括烤烟、混合型卷烟和雪茄烟。其中，“黄山”是具有60余年发展历史、拥有“中国驰名商标”称号、销量排名全国第8位的大品牌；“都宝”是国内混合型卷烟的代表品牌，是中国烟草进军海外市场的重点品牌；手工雪茄烟占全国同类产品42%的市场份额，是国产手工雪茄烟重要品牌。

[Sub-question query engine] Q: 浙江中烟的核心产品是什么
[Sub-question query engine] A: 浙江中烟的核心产品是卷烟，其品牌有利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。浙江中烟工业有限责任公司是中国烟草总公司的全资子公司，注册资本为人民币97600万元，主要经营卷烟生产销售业务。

根据已知信息，安徽中烟的核心产品包括烤烟、混合型卷烟和雪茄烟，其中“黄山”、“都宝”和手工雪茄烟是其重要品牌。浙江中烟的核心产品是卷烟，其品牌有利群、大红鹰、雄狮、上游、五一、双叶、西湖、新安江。安徽中烟的产品范围较浙江中烟广泛，涵盖了烤烟、混合型卷烟和雪茄烟等多种类型。浙江中烟主要经营卷烟生产销售业务，品牌较少，主要集中在卷烟领域。</已知信息>
---------------------



In [173]:
response_3.metadata["selector_result"].selections[0].reason

'The question asks to compare two things (安徽中烟 and 浙江中烟) and their core products. This can be effectively broken down into simpler sub-questions, each of which can be answered independently.'